In [32]:
import xarray as xr
import numpy as np


In [ ]:

weatherbench = xr.open_zarr("gs://weatherbench2/datasets/era5/1959-2023_01_10-full_37-1h-0p25deg-chunk-1.zarr")
weatherbench

In [ ]:


P_level_13 = [
    50, 100, 150, 200, 250, 
    300, 400, 500, 600, 700, 
    850, 925, 1000
]

PRESSURE_VARIABLES=[
    "u_component_of_wind",
    "v_component_of_wind",
    "specific_humidity",
    "temperature",
    "vertical_velocity",
    "geopotential"
]

SURFACE_VARIABLES=[
    "10m_u_component_of_wind", 
    "10m_v_component_of_wind", 
    "mean_sea_level_pressure",
    "sea_surface_temperature",
    "2m_temperature",
    "geopotential_at_surface",
    "land_sea_mask"
]

from datetime import date, timedelta

# will be iterated over 52 weeks
date = date(2021, 1, 4)
date_before = date - timedelta(days=1)

In [35]:
import pandas as pd

start_date = '2021-01-01'
end_date = '2021-12-31'
date_range = pd.date_range(start=start_date, end=end_date, freq='W-MON')

for date in date_range:
    date_before = date - timedelta(days=1)
    
    print (f"date: {date}")
    print (f"date_before: {date_before}")
    

date: 2021-01-04 00:00:00
date_before: 2021-01-03 00:00:00
date: 2021-01-11 00:00:00
date_before: 2021-01-10 00:00:00
date: 2021-01-18 00:00:00
date_before: 2021-01-17 00:00:00
date: 2021-01-25 00:00:00
date_before: 2021-01-24 00:00:00
date: 2021-02-01 00:00:00
date_before: 2021-01-31 00:00:00
date: 2021-02-08 00:00:00
date_before: 2021-02-07 00:00:00
date: 2021-02-15 00:00:00
date_before: 2021-02-14 00:00:00
date: 2021-02-22 00:00:00
date_before: 2021-02-21 00:00:00
date: 2021-03-01 00:00:00
date_before: 2021-02-28 00:00:00
date: 2021-03-08 00:00:00
date_before: 2021-03-07 00:00:00
date: 2021-03-15 00:00:00
date_before: 2021-03-14 00:00:00
date: 2021-03-22 00:00:00
date_before: 2021-03-21 00:00:00
date: 2021-03-29 00:00:00
date_before: 2021-03-28 00:00:00
date: 2021-04-05 00:00:00
date_before: 2021-04-04 00:00:00
date: 2021-04-12 00:00:00
date_before: 2021-04-11 00:00:00
date: 2021-04-19 00:00:00
date_before: 2021-04-18 00:00:00
date: 2021-04-26 00:00:00
date_before: 2021-04-25 00:00:

In [30]:
pres_surf = weatherbench[
    PRESSURE_VARIABLES + SURFACE_VARIABLES
    ].sel(
        level=P_level_13, 
        time=[f"{date}T00:00:00", f"{date}T12:00:00"]
        )

tp = weatherbench[
    "total_precipitation"
    ].sel(
        time=slice(f"{date_before}T13:00:00", f"{date}T12:00:00")
    ).resample(time="12h", 
    closed="right", 
    label="right").sum()

input = xr.merge(
    [pres_surf, tp]
    ).expand_dims(
        dim={'batch': [0]}
    ).rename({
        "latitude":"lat",
        "longitude":"lon"
    })

for var in ['geopotential_at_surface', 'land_sea_mask']:
    if 'batch' in input[var].dims:
        input[var] = input[var].squeeze('batch')

input.to_netcdf("input.nc")

In [25]:
xr.open_dataset("/geodata2/Gencast/input/2025-01-02.nc")

<xarray.Dataset> Size: 44MB
Dimensions:                   (batch: 1, time: 2, lat: 181, lon: 360, level: 13)
Coordinates:
  * batch                     (batch) int64 8B 0
  * time                      (time) timedelta64[ns] 16B 00:00:00 12:00:00
  * lat                       (lat) float64 1kB -90.0 -89.0 -88.0 ... 89.0 90.0
  * lon                       (lon) float64 3kB 0.0 1.0 2.0 ... 358.0 359.0
  * level                     (level) int32 52B 50 100 150 200 ... 850 925 1000
    datetime                  (batch, time) datetime64[ns] 16B ...
Data variables: (12/14)
    10m_u_component_of_wind   (batch, time, lat, lon) float32 521kB ...
    10m_v_component_of_wind   (batch, time, lat, lon) float32 521kB ...
    2m_temperature            (batch, time, lat, lon) float32 521kB ...
    mean_sea_level_pressure   (batch, time, lat, lon) float32 521kB ...
    sea_surface_temperature   (batch, time, lat, lon) float32 521kB ...
    geopotential_at_surface   (lat, lon) float32 261kB ...
    ...                        ...
    geopotential              (batch, time, level, lat, lon) float32 7MB ...
    specific_humidity         (batch, time, level, lat, lon) float32 7MB ...
    temperature               (batch, time, level, lat, lon) float32 7MB ...
    u_component_of_wind       (batch, time, level, lat, lon) float32 7MB ...
    v_component_of_wind       (batch, time, level, lat, lon) float32 7MB ...
    vertical_velocity         (batch, time, level, lat, lon) float32 7MB ...
Attributes:
    GRIB_edition:            1
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-01-24T14:08 GRIB to CDM+CF via cfgrib-0.9.1...